In [1]:
import streamlit as st
import os
import base64
import json
import datetime
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import streamlit as st
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import base64
import os
import random
import string
import pdfplumber
from dotenv import load_dotenv, find_dotenv
import pandas as pd
from langchain.chat_models import AzureChatOpenAI
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text
import datetime
from datetime import timezone
import webbrowser



ModuleNotFoundError: No module named 'streamlit'

In [2]:
# Constants
MAX_TOKENS = 4096
TEMPERATURE = 0.7

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

processed_emails = set()

In [ ]:
def authenticate_gmail():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

In [ ]:
def get_today_emails_with_attachments(service):
    today = datetime.datetime.now().strftime('%Y/%m/%d')
    query = f'has:attachment after:{today}'
    results = service.users().messages().list(userId='me', q=query).execute()
    messages = results.get('messages', [])
    return messages

In [ ]:
def get_message(service, msg_id):
    return service.users().messages().get(userId='me', id=msg_id).execute()

In [ ]:
def download_attachment(service, msg_id, store_dir='downloads'):
    message = get_message(service, msg_id)
    for part in message['payload']['parts']:
        if part['filename']:
            if 'data' in part['body']:
                data = part['body']['data']
            else:
                att_id = part['body']['attachmentId']
                att = service.users().messages().attachments().get(userId='me', messageId=msg_id, id=att_id).execute()
                data = att['data']
            file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
            path = os.path.join(store_dir, part['filename'])
            with open(path, 'wb') as f:
                f.write(file_data)
            return path

In [ ]:
def parse_pdf(path):
    try:
        with pdfplumber.open(path) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        st.error(f"Failed to extract text from PDF: {e}")
        return ""

In [ ]:
def main():
    st.title("Gmail Attachment Monitor")
    _ = load_dotenv(find_dotenv())
    llm = AzureChatOpenAI(
            openai_api_type=os.environ['OPENAI_API_TYPE'],
            openai_api_base=os.environ['OPENAI_API_BASE'],
            openai_api_version=os.environ['OPENAI_API_VERSION'],
            deployment_name=os.environ['OPENAI_API_DEPLOYMENT'],
            model_name=os.environ['OPENAI_API_MODEL'],
            openai_api_key=os.environ['OPENAI_API_KEY'],
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS
        )
    
    service = authenticate_gmail()
    st.write("Authenticated with Gmail")

    check_emails = st.button("Check for new emails")

    if check_emails:
        messages = get_today_emails_with_attachments(service)
        for msg in messages:
            msg_id = msg['id']
            if msg_id not in processed_emails:
                message = get_message(service, msg_id)
                subject = next(header['value'] for header in message['payload']['headers'] if header['name'] == 'Subject')
                email = next(header['value'] for header in message['payload']['headers'] if header['name'] == 'From')

                attachment_path = download_attachment(service, msg_id)
                parsed_content_from_pdf = parse_pdf(attachment_path)
                parsed_content= parsingfunction(llm,parsed_content_from_pdf)

                if not parsed_content:
                    st.warning(f"Email '{subject}' does not contain valid parsed content.")
                    continue  # Skip processing further

                with st.expander(f"Email: {subject} from {email}"):
                        st.write(f"Subject: {subject}")

                        # Displaying tables for each parsed content section
                        if 'invoice' in parsed_content:
                            st.subheader("Invoice Extraction")
                            st.table(pd.DataFrame(parsed_content['invoice']['invoice_extraction']))

                        if 'remit_address' in parsed_content:
                            st.subheader("Remit Address")
                            st.table(pd.DataFrame(parsed_content['remit_address']['remit_address']))

                        if 'billing_address' in parsed_content:
                            st.subheader("Billing Address")
                            st.table(pd.DataFrame(parsed_content['billing_address']['billing_address']))

                        if 'shipping_address' in parsed_content:
                            st.subheader("Shipping Address")
                            st.table(pd.DataFrame(parsed_content['shipping_address']['shipping_address']))

                        if 'products' in parsed_content:
                            st.subheader("Products")
                            st.table(pd.DataFrame(parsed_content['products']['bill']))

                        if st.button(f"View {subject}", key=f"view_{msg_id}"):
                             webbrowser.open(f"file://{os.path.abspath(attachment_path)}")

                    # Mark this email as processed
                processed_emails.add(msg_id)

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
def parsingfunction(llm,processed_text):
    invoice_schema = Object(
            id="invoice_extraction",
            description="extraction of relevant information from invoice",
            attributes=[
                Text(
                    id="invoice_number",
                    description="unique number (identifier) of given invoice",
                    examples=[
                        ("1019273", "1019273"),
                        ("291870.4-1", "291870.4-1"),
                        ("291870.5-1", "291870.5-1")
                    ]
                ),
                Text(
                    id="invoice_date",
                    description="date of the invoice",
                    examples=[
                        ("12/20/2023", "12/20/2023"),
                        ("2024-05-27", "2024-05-27")
                    ]
                ),
                Text(
                    id="customer_id",
                    description="identifier for the customer",
                    examples=[
                        ("47900", "47900"),
                        ("XT5570", "XT5570")
                    ]
                ),
                Text(
                    id="customer_po",
                    description="customer purchase order number",
                    examples=[
                        ("832970", "832970"),
                        ("838290", "838290"),
                        ("838940", "838940")
                    ]
                ),
                Text(
                    id="salesperson",
                    description="salesperson handling the invoice",
                    examples=[
                        ("James Flemming", "James Flemming"),
                        ("Elaine Kirby", "Elaine Kirby")
                    ]
                ),
                Text(
                    id="terms",
                    description="payment terms",
                    examples=[
                        ("Net 30", "Net 30"),
                        ("NET 45", "NET 45")
                    ]
                ),
                Text(
                    id="due_date",
                    description="due date for payment",
                    examples=[
                        ("1/19/2024", "1/19/2024"),
                        ("Jul 12, 2024", "Jul 12, 2024")
                    ]
                ),
                Text(
                    id="packing_slip_no",
                    description="packing slip number",
                    examples=[
                        ("20369", "20369")
                    ]
                ),
                Text(
                    id="tracking_number",
                    description="tracking number for the shipment",
                    examples=[
                        ("1Z1R7W540343315285", "1Z1R7W540343315285"),
                        ("1Z8568830390831159", "1Z8568830390831159"),
                        ("1Z8868940261123687", "1Z8868940261123687"),
                        ("1Z8568830391553949", "1Z8568830391553949")
                    ]
                )
            ],
            many=False,
        )

    billing_address_schema = Object(
            id="billing_address",
            description="where the bill for a product or service is sent so it can be paid by the recipient, also known as Bill to or Authorized by",
            attributes=[
                Text(id="name", description="the name of person and organization"),
                Text(id="address_line", description="the local delivery information such as street, building number, PO box, or apartment portion of a postal address"),
                Text(id="city", description="the city portion of the address"),
                Text(id="state_province_code", description="the code for address US states"),
                Text(id="postal_code", description="the postal code portion of the address"),
                Text(id="country", description="the country portion of the address")
            ],
            examples=[
                (
                    "B I L L T O TRIMBLE NAVIGATION LTD - SUNNYVALE 5475 KELLENBURGER ROAD DAYTON OH 45424 United States of America",
                    {
                        "name": "TRIMBLE NAVIGATION LTD - SUNNYVALE",
                        "address_line": "5475 KELLENBURGER ROAD",
                        "city": "DAYTON",
                        "state_province_code": "OH",
                        "postal_code": "45424",
                        "country": "United States of America"
                    }
                ),
                (
                    "Bill To Trimble Navigation Sunnyvale & Dayton AP 5475 Kellenburger Road Dayton OH 45424-1099 USA",
                    {
                        "name": "Trimble Navigation Sunnyvale & Dayton AP",
                        "address_line": "5475 Kellenburger Road",
                        "city": "Dayton",
                        "state_province_code": "OH",
                        "postal_code": "45424-1099",
                        "country": "USA"
                    }
                )
            ],
            many=False,  # This indicates multiple billing addresses can be extracted
        )


    remit_address_schema = Object(
            id="remit_address",
            description="where the product or service is remit to",
            attributes=[
                Text(id="name", description="the name of person and organization"),
                Text(id="address_line", description="the local delivery information such as street, building number, PO box, or apartment portion of a postal address"),
                Text(id="city", description="the city portion of the address"),
                Text(id="state_province_code", description="the code for address US states"),
                Text(id="postal_code", description="the postal code portion of the address"),
                Text(id="country", description="the country portion of the address")
            ],
            examples=[
                (
                    "Remit To Amphenol DC Electronics 28704 Network Place Chicago, IL 60673-1286",
                    {
                        "name": "Amphenol DC Electronics",
                        "address_line": "28704 Network Place",
                        "city": "Chicago",
                        "state_province_code": "IL",
                        "postal_code": "60673-1286",
                        "country": "USA"
                    }
                ),
                (
                    "TRIMBLE NAVIGATION LTD 4450 GIBSON DRIVE TIPP CITY OH 45371 United States of America Phone: 650-593-3288 Remit To: ",
                    {
                        "name": "TRIMBLE NAVIGATION LTD",
                        "address_line": "4450 GIBSON DRIVE",
                        "city": "TIPP CITY",
                        "state_province_code": "OH",
                        "postal_code": "45371",
                        "country": "United States of America"
                    }
                ),
                (
                    "Remit To: TRIMBLE NAVIGATION LTD 4450 GIBSON DRIVE TIPP CITY OH 45371 United States of America",
                    {
                        "name": "TRIMBLE NAVIGATION LTD",
                        "address_line": "4450 GIBSON DRIVE",
                        "city": "TIPP CITY",
                        "state_province_code": "OH",
                        "postal_code": "45371",
                        "country": "United States of America"
                    }
                )
            ],
            many=True  # Only one remit address should be extracted
        )


    shipping_address_schema = Object(
            id="shipping_address",
            description="where the product or service is shipped",
            attributes=[
                Text(id="name", description="the name of person and organization"),
                Text(id="address_line", description="the local delivery information such as street, building number, PO box, or apartment portion of a postal address"),
                Text(id="city", description="the city portion of the address"),
                Text(id="state_province_code", description="the code for address US states"),
                Text(id="postal_code", description="the postal code portion of the address"),
                Text(id="country", description="the country portion of the address")
            ],
            examples=[
                (
                    "SHIP TO TRIMBLE NAVIGATION LTD 4450 GIBSON DRIVE TIPP CITY OH 45371 United States of America",
                    {
                        "name": "TRIMBLE NAVIGATION LTD",
                        "address_line": "4450 GIBSON DRIVE",
                        "city": "TIPP CITY",
                        "state_province_code": "OH",
                        "postal_code": "45371",
                        "country": "United States of America"
                    }
                ),
                (
                    "Ship To Amphenol DC Electronics 28704 Network Place Chicago, IL 60673-1286",
                    {
                        "name": "Amphenol DC Electronics",
                        "address_line": "28704 Network Place",
                        "city": "Chicago",
                        "state_province_code": "IL",
                        "postal_code": "60673-1286",
                        "country": "USA"
                    }
                )
                ,
                (
                    "SHIP TO TRIMBLE NAVIGATION LTD 4000 GIBSON DRIVE LAS CITY OH 45871 United States of America",
                    {
                        "name": "TRIMBLE NAVIGATION LTD",
                        "address_line": "4000 GIBSON DRIVE",
                        "city": "LAS CITY",
                        "state_province_code": "OH",
                        "postal_code": "45871",
                        "country": "United States of America"
                    }

                )
            ],
            many=False,  # This indicates multiple shipping addresses can be extracted
        )

    products_schema = Object(
            id="bill",
            description="the details of bill",
            attributes=[
                Text(id="product_description", description="the description of the product or service"),
                Text(id="count", description="number of units bought for the product"),
                Text(id="unit_item_price", description="price per unit"),
                Text(id="product_total_price", description="the total price, which is number of units * unit_price"),
            ],
            examples=[
                (
                    "SC8702 3/8-16X2 HX-CAP-SC-GR8-Z-YEL - 132091 100 0.90000 90.00",
                    {
                        "product_description": "SC8702 3/8-16X2 HX-CAP-SC-GR8-Z-YEL - 132091",
                        "count": 100,
                        "unit_item_price": 0.90000,
                        "product_total_price": 90.00,
                    }
                ),
                (
                    "CABLE, GNSS ANTENNA, 20 M 40 $197.58 $7,903.20",
                    {
                        "product_description": "CABLE, GNSS ANTENNA, 20 M",
                        "count": 40,
                        "unit_item_price": 197.58,
                        "product_total_price": 7903.20,
                    }
                ),
                (
                    "CABLE, RADIO ANTENNA, UHF, TNC TO N, LMR400-UF 10 $90.93 $909.30",
                    {
                        "product_description": "CABLE, RADIO ANTENNA, UHF, TNC TO N, LMR400-UF",
                        "count": 10,
                        "unit_item_price": 90.93,
                        "product_total_price": 909.30,
                    }
                ),
                (
                    "POLE,EXTENSION,1ft,TNL REV A 50 15.91000 795.50",
                    {
                        "product_description": "POLE,EXTENSION,1ft,TNL REV A",
                        "count": 50,
                        "unit_item_price": 15.91,
                        "product_total_price": 795.50,
                    }
                )
            ],
            many=True  # This indicates multiple products can be extracted
        )
    invoice_chain = create_extraction_chain(llm, invoice_schema)
    remit_address_chain = create_extraction_chain(llm, remit_address_schema)
    billing_address_chain = create_extraction_chain(llm, billing_address_schema)
    shipping_address_chain = create_extraction_chain(llm, shipping_address_schema)
    products_chain = create_extraction_chain(llm, products_schema)

    invoice = invoice_chain.predict_and_parse(text=processed_text)["data"]
    remit_address = remit_address_chain.predict_and_parse(text=processed_text)["data"]
    billing_address = billing_address_chain.predict_and_parse(text=processed_text)["data"]
    shipping_address = shipping_address_chain.predict_and_parse(text=processed_text)["data"]
    products = products_chain.predict_and_parse(text=processed_text)["data"]

    extracted_data = {
        "invoice": invoice,
        "remit_address": remit_address,
        "billing_address": billing_address,
        "shipping_address": shipping_address,
        "products": products
    }

    return extracted_data
